# Mediator

### 1. Pull in dependencies

In [ ]:
%autoawait
import time
import asyncio

### 2. Instatiate the controller for our Agent

The arguments depend on how the aca-py agent was initiated. See the manage and docker-compose.yml files for more details.

In [ ]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8042
WEBHOOK_BASE = ""
ADMIN_URL = "http://mediator-agent:8041"


# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)


### 3. Listen for webhooks and register default listeners

TODO: Explore mediator webhook stuff

In [ ]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

### 4. Use the controller to create an invitation from our agent

An invitation is a JSON object, as shown below, it contains the relevant information required for another agent to connect with it and exchange identifiers. This object must be passed to the agent Alice wishes to connect to out of band, in this instance we will just copy it across.

In [ ]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(multi_use="true")
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

### Copy the invitation output to any agent that you needs to establish a mediator. For example where you came from - [Alice](http://localhost:8888/notebooks/Alice/Part%202%20-%20Mediation%20of%20communication%20-%20Alice.ipynb)

### Check Mediation Records

In [ ]:
response = await agent_controller.mediation.get_mediation_records()

for record in response:
    print("Mediation Record")
    print("connection_id", record["connection_id"])
    print("State", record["state"])

## Terminate Meditator Controller

**Only do this when you have completed the full tutorial**

In [ ]:
response = await agent_controller.terminate()
print(response)